### Integral with constant key (super Toy)

In [20]:
# Define the finite field GF(2^5) with generator a.
F.<a> = GF(2^5)
print("Irreducible polynomial:", F.modulus())

# --- MiMC encryption functions ---

def mimc_round(x):
    """
    One round of MiMC: add constant 1 and cube.
    """
    return (x + 1)**3

def two_round_mimc(x):
    """
    Two rounds of MiMC encryption.
    """
    return mimc_round(mimc_round(x))

# --- Integral over a subspace ---

def integral_attack_two_round_mimc(base, subspace_basis=None):
    """
    Given a base element in GF(2^5), compute the sum of the two-round MiMC encryption
    over an affine subspace of dimension 3. That is, if U is a 3-dimensional subspace of GF(2^5)
    and base is an offset, then compute:
    
         S(base) = sum_{u in U} f(base + u),
         
    where f is the two-round MiMC function.
    
    Parameters:
      base : an element of GF(2^5) representing the offset of the affine subspace.
      subspace_basis : a list of 3 elements of GF(2^5) forming a basis for a subspace.
                       If None, defaults to [1, a, a^2].
    
    Returns:
      The sum S(base) in GF(2^5) of f(x) over x in (base + U).
    """
    # Default subspace basis: span{1, a, a^2} (they are linearly independent in GF(2^5)).
    if subspace_basis is None:
        subspace_basis = [F(1), a, a^2]
    
    # Construct the subspace U = { c0*v0 + c1*v1 + c2*v2 : c_i in GF(2) }.
    U = []
    for c0 in [F(0), F(1)]:
        for c1 in [F(0), F(1)]:
            for c2 in [F(0), F(1)]:
                U.append(c0 * subspace_basis[0] + c1 * subspace_basis[1] + c2 * subspace_basis[2])
    
    # Compute the sum over the affine subspace: base + U.
    s = F(0)
    for u in U:
        print(base,u)
        print(s)
        s += two_round_mimc(base + u)
    return s

# --- Example usage ---
# Choose a base element (for example, 0) and compute the sum over the affine subspace.
base = F(0)
result = integral_attack_two_round_mimc(base)
print("\nSum over affine subspace (base = %s): %s" % (base, result))


Irreducible polynomial: x^5 + x^2 + 1
0 0
0
0 a^2
0
0 a
a^4 + 1
0 a^2 + a
a^4 + a^2
0 1
a^3 + a + 1
0 a^2 + 1
a^3 + a
0 a + 1
a^3 + a^2
0 a^2 + a + 1
a^4 + a^2 + 1

Sum over affine subspace (base = 0): 0


### Integral attack with variable key 

not super flexible (for example for more than 3 rounds you need reduct degree in key)

In [33]:
# Define the finite field GF(2^5) with generator a.
F.<a> = GF(2^5)
print("Irreducible polynomial:", F.modulus())

########################################################################
# --- Symbolic MiMC encryption with variable round constants ---

# Define a polynomial ring for the round constants over F.
# For example, for 3 rounds we introduce three indeterminates k1, k2, k3.
Rk.<k1, k2, k3, k4> = PolynomialRing(F)

# Now define a polynomial ring in x over Rk.
# In this ring, the coefficients are polynomials in k1,k2,k3.
P.<x> = PolynomialRing(Rk)
reduction_poly = x**(2**5) - x
def mimc_round_symbolic(x, k):
    """
    One round of MiMC: compute (x + k)^3.
    x is an element of P (a polynomial in x), and k is a symbolic constant.
    """
    return (x + k)**3 % reduction_poly

def iterated_mimc_symbolic(x, k_list):
    """
    Compute the iterated MiMC encryption symbolically.
    
    Parameters:
      x : an element of P.
      k_list : a list of symbolic round constants (elements of Rk).
    
    Returns:
      The symbolic expression f(x) = R_k3(R_k2(R_k1(x))) in P.
    """
    for k in k_list:
        x = mimc_round_symbolic(x, k)
    return x

# Build a symbolic expression for a 3-round MiMC function.
f_symbolic = iterated_mimc_symbolic(x, [k1, k2, k3])
print("\nSymbolic expression for 3-round MiMC with variable round constants:")
print(f_symbolic)

########################################################################
# --- Integral Attack over an affine subspace (dimension 3) ---

def integral_attack_symbolic(base, k_list, subspace_basis=None):
    """
    For a given base element in GF(2^5) and a list of symbolic round constants,
    compute the sum over an affine subspace of dimension 3 of the symbolic MiMC encryption.
    
    More precisely, let U be a 3-dimensional subspace of GF(2^5) (by default, spanned by {1, a, a^2}).
    For a given base b, we compute
       S(b) = sum_{u in U} f(b + u)
    where f(x) is the iterated MiMC encryption function (symbolically defined).
    
    Parameters:
      base : an element of F (the offset).
      k_list : list of symbolic constants to be used in iterated_mimc_symbolic.
      subspace_basis : a list of 3 elements in F forming a basis for U. If None, defaults to [1, a, a^2].
    
    Returns:
      A symbolic expression (in P) representing the sum S(b) in GF(2^5) with coefficients in Rk.
    """
    # Default subspace basis: span{1, a, a^2} in F.
    if subspace_basis is None:
        subspace_basis = [F(1), a, a^2]
    
    # Construct the 3-dimensional subspace U = { c0*v0 + c1*v1 + c2*v2 : c_i in GF(2) }.
    U = []
    for c0 in [F(0), F(1)]:
        for c1 in [F(0), F(1)]:
            for c2 in [F(0), F(1)]:
                U.append(c0 * subspace_basis[0] + c1 * subspace_basis[1] + c2 * subspace_basis[2])
    
    # Sum over the affine subspace: base + U.
    s = P(0)   # 0 in P (constant zero)
    for u in U:
        # Convert base+u (which is in F) into a constant element of P.
        s += iterated_mimc_symbolic(P(base + u), k_list)
        #print(f'Input: {u}, output: {iterated_mimc_symbolic(P(base + u), k_list)}')
    return s

# --- Example usage of the symbolic integral attack ---

# Choose a base element in F, for example 0.
base = F(0)
# Use the same symbolic constants as defined: [k1, k2, k3].
sym_integral = integral_attack_symbolic(base, [k1, k2, k3], [a^3, a^4, a^2])
print("\nSymbolic sum over affine subspace (base =", base, "):")
print(sym_integral)


Irreducible polynomial: x^5 + x^2 + 1

Symbolic expression for 3-round MiMC with variable round constants:
x^27 + k1*x^26 + k1^2*x^25 + (k1^3 + k2)*x^24 + k1^8*x^19 + (k1^9 + k3)*x^18 + k1^10*x^17 + (k1^11 + k1^8*k2 + k1^2*k3)*x^16 + k2^2*k3*x^12 + k1^16*x^11 + k1^17*x^10 + (k1^18 + k3^2)*x^9 + (k1^19 + k1^16*k2 + k1^4*k2^2*k3 + k1*k3^2)*x^8 + (k2^4*k3 + k2*k3^2)*x^6 + (k1^8*k2^2*k3 + k1^2*k2^4*k3 + k1^2*k2*k3^2)*x^4 + (k1^24 + k2^8 + k2^2*k3^2)*x^3 + (k1^25 + k1^16*k3 + k1*k2^8 + k1^4*k2^4*k3 + k1^4*k2*k3^2 + k1*k2^2*k3^2)*x^2 + (k1^26 + k1^2*k2^8 + k1^8*k3^2 + k1^2*k2^2*k3^2)*x + k1^27 + k1^24*k2 + k1^18*k3 + k1^12*k2^2*k3 + k1^3*k2^8 + k1^6*k2^4*k3 + k1^9*k3^2 + k2^9 + k1^6*k2*k3^2 + k2^6*k3 + k1^3*k2^2*k3^2 + k2^3*k3^2 + k3^3

Symbolic sum over affine subspace (base = 0 ):
k1^16 + (a^4)*k1^8 + (a^2)*k1^2 + k1 + (a^3 + 1)
